<img src="../img/GTK_Logo_Social Icon.jpg" width=175 align="right" />


# Worksheet 3: Approximate String Matching - Answers

This worksheet covers concepts relating to approximate string matching.  It should take no more than 20-30 minutes to complete.  Please raise your hand if you get stuck.  

There are many ways to accomplish the tasks that you are presented with, however you will find that by using the techniques covered in class, the exercises should be relatively simple. 

## Import the Libraries
For this exercise, we will be using:
* Pandas (http://pandas.pydata.org/pandas-docs/stable/)
* FuzzyWuzzy (https://pypi.org/project/fuzzywuzzy/)
* TLDExtract (https://pypi.org/project/tldextract/)

In [4]:
# Run this cell first
import tldextract
import pandas as pd
from fuzzywuzzy import fuzz

In [5]:
# Read in the data
df = pd.read_csv('../data/url_log.csv')

## Exercise 1:  Find Spoofed Domains
For this exercise, we gave you some notional log data.  The scenario is that you work for Microsoft and have reports of people getting hacked by malicious false Microsoft sites.

Your challenge: Find these Spoofers.
First, explore the data a bit:

In [6]:
# First explore the data a bit
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1005 non-null   int64 
 1   src_ip  1005 non-null   object
 2   url     1005 non-null   object
 3   bytes   1005 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 31.5+ KB


In [7]:
df.sample(5)

,id,src_ip,url,bytes
841,1679429128000,228.38.78.24,https://admin.ch/risus/praesent/lectus/vestibu...,1601
988,1679385315000,190.133.41.12,http://sciencedaily.com/morbi/non/lectus/aliqu...,3689
747,1679424449000,185.40.64.135,https://quantcast.com/penatibus/et/magnis/dis/...,2462
230,1679435723000,102.148.207.11,http://bizjournals.com/ac/nulla/sed/vel/enim/s...,2472
179,1679435788000,185.54.251.114,https://europa.eu/praesent/blandit/nam.html?nu...,3291


### Step Two:  Extract Artifacts
In order to find the spoofed domains, we will be using an approximate string distance function to find the spoofed domains.  We'll be using the Levenshtein Distance to find domains that are similar to the legitimate domain.  For instance, micr0soft.com.  You can read more about the Levenshtein Distance here: https://en.wikipedia.org/wiki/Levenshtein_distance.  A score of 100 is a perfect match, and 0 is not a match at all. 

We will use FuzzyWuzzy to calculate the distance. There is a link above to the documentation for FuzzyWuzzy. 

What we're going to do first is write two functions:

* `get_domain(url)`:  This function will take a complete URL and extract the domain portion of the URL.
* `similiarity_score(x,y)`:  This function will accept two strings and return the levenshtein distance between the two strings.


In [20]:
def get_domain(url: str) -> str:
    parts = tldextract.extract(url)
    return f"{parts.domain}.{parts.suffix}"

def similarity_score(x:str,y:str) -> str:
    return fuzz.ratio(x, y)

Now that you have the functions written, let's add two columns to the data frame:
1. `domain`:  This should only contain the domain portion of the URL
2. `score`:  The Levenshtein distance between the domain and microsoft.com

In [21]:
df['domain'] = df['url'].apply(get_domain)

In [23]:
df['score'] = df.apply(lambda x: similarity_score(x['domain'], 'microsoft.com'),axis=1)

## Step Three:  Find the Bad Guys!
Now that you have the data in the dataframe, see if you can find interesting permutations of the microsoft domain, by looking for different score thresholds.  What did you find?

In [25]:
df[df['score'] > 80]

,id,src_ip,url,bytes,domain,score
19,1679385540000,133.141.41.150,https://microsoft.com/condimentum/id/luctus.jp...,2605,microsoft.com,100
241,1679394372000,154.239.156.192,http://miсrosoft.com/est/phasellus/sit/amet/er...,1385,miсrosoft.com,92
296,1679440341000,223.243.164.245,https://microsoft.com/habitasse/platea/dictums...,2038,microsoft.com,100
303,1679382550000,22.207.178.188,http://microsoft.com/quis/orci/eget/orci/vehic...,561,microsoft.com,100
315,1679394372000,154.239.156.192,http://miсrosoft.com/est/phasellus/sit/amet/er...,1385,miсrosoft.com,92
321,1679394372000,154.239.156.192,http://miсrosoft.com/est/phasellus/sit/amet/er...,1385,miсrosoft.com,92
379,1679426706000,45.175.213.176,http://microsoft.com/eget.jsp?quis=porta,2887,microsoft.com,100
810,1679410900000,145.24.30.31,http://microsoft.com/blandit/mi/in/porttitor/p...,3916,microsoft.com,100
